# Non Anomolous Training Sample Creation

 Vinayak Sharma

 Non Anomolous Sample Creation for training - MSS Dish 5g

 Notebook will try and create and find usable datasets that can be trained on by anomoly detection models to learn what non anomolous behavior looks like

----

## Defining Anomolous Data

Since non-anomolous data can have a range of implications per dataset and per use case, we will try and find samples of data on a per cluster basis that fit the following criteria

Non Anomolous Data will have the following properties in each of the following rec types:


cluster_failed_node_count == 0


node_memory_failcnt  == 0 <br />
node_memory_hierarchical_pgfault == 0<br />
node_memory_hierarchical_pgmajfault == 0<br />
node_memory_pgfault ==0<br />
node_memory_pgmajfault == 0<br />
node_network_rx_dropped ==0<br />
node_network_rx_errors  == 0<br />
node_network_tx_dropped ==0<br />
node_network_tx_errors ==0<br />


pod_memory_failcnt == 0<br />
pod_memory_hierarchical_pgfault == 0<br />
pod_memory_hierarchical_pgmajfault == 0<br />
pod_memory_pgfault == 0<br />
pod_memory_pgmajfault == 0<br />
pod_network_rx_dropped == 0<br />
pod_network_rx_errors == 0<br />
pod_network_tx_dropped == 0 <br />
pod_network_tx_errors == 0<br />
pod_number_of_container_restarts < ?<br />
pod_status != Failed<br />



container_memory_failcnt == 0<br />
container_memory_hierarchical_pgfault == 0<br />
container_memory_hierarchical_pgmajfault == 0<br />
container_memory_pgfault == 0<br />
container_memory_pgmajfault == 0<br />





-----

Note: Need to find the healthy pod number of container restarts

-----

## Expiriment 1: Lets try and create a methadology to find non-anomolous samples

### Trail 1: Lets start with one hour of data, and isolate only healthy samples as per the definition above

First things first lets read in some data

In [2]:
from msspackages import update_cwd_to_root
update_cwd_to_root()


current repository: /root/understanding-eks-data/research
current repository: /root/understanding-eks-data


In [3]:
##Imports
from data_ingestion import Pyspark_data_ingestion
import pandas as pd
import numpy as np
import random
from tqdm import tqdm 

#For plotting
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pyspark.sql.functions as func
from pyspark.sql.functions import col, countDistinct
from datetime import timedelta

In [4]:
samples = [('2022', '5', '5', -1)]
rec_types = ["Cluster", "Node", "Pod", "Container"]

In [5]:
%%time
df_cluster_build = Pyspark_data_ingestion(*samples[0], rec_types[0],setup = '128gb')
err1,df_cluster = df_cluster_build.read()
df_cluster = df_cluster.toPandas()

CPU times: user 9.17 s, sys: 226 ms, total: 9.4 s
Wall time: 3min 54s


In [6]:
%%time
df_node_build = Pyspark_data_ingestion(*samples[0], rec_types[1],setup = '128gb')
err2,df_node = df_node_build.read()
df_node = df_node.toPandas()

CPU times: user 1min 41s, sys: 3.84 s, total: 1min 45s
Wall time: 5min 16s


In [7]:
%%time
df_pod_build = Pyspark_data_ingestion(*samples[0], rec_types[2],setup = '128gb')
err3,df_pod = df_pod_build.read()
df_pod = df_pod.toPandas()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.7/site-packages/py4j/clientserver.py", line 540, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/py4j/

ValueError: Unable to parse datatype from schema. An error occurred while calling o347.schema

In [8]:
%%time
df_container_build = Pyspark_data_ingestion(*samples[0], rec_types[3],setup = '128gb')
err4,df_container = df_container_build.read()
df_container = df_container.toPandas()

ConnectionRefusedError: [Errno 111] Connection refused

In [9]:

# %%time

# #use pyspark create objects able to read in data
# df_cluster_build = Pyspark_data_ingestion(*samples[0], rec_types[0],setup = '128gb')

# df_node_build = Pyspark_data_ingestion(*samples[0], rec_types[1],setup = '128gb')

# df_pod_build = Pyspark_data_ingestion(*samples[0], rec_types[2],setup = '128gb')

# df_container_build = Pyspark_data_ingestion(*samples[0], rec_types[3],setup = '128gb')



# # use the pyspark objects to actually read in data
# err1,df_cluster = df_cluster_build.read()
# err2,df_node = df_node_build.read()
# err3,df_pod = df_pod_build.read()
# err4,df_container = df_container_build.read()



# # convert the pyspark dataframes to pandas

# df_cluster = df_cluster.toPandas()
# df_node = df_node.toPandas()
# df_pod = df_pod.toPandas()
# df_container = df_container.toPandas()



Next we will find all of the cluster names in which any Nodes,Pods,Containers may be showing failures/faults

In [ ]:
##Filter by cluster, node, pod, and container all of the indexes that have anomolous behavior in them. We will later on purge our datasets of the clusters that behave in these manners to train on clean data

filter_Condition_Cluster = np.where( df_cluster['cluster_failed_node_count']>0 ) 



filter_Condition_Node = np.where( 
                                    (df_node['node_memory_failcnt']>0) |
                                    (df_node['node_memory_hierarchical_pgfault']>0) |
                                    (df_node['node_memory_hierarchical_pgmajfault']>0)|
                                    (df_node['node_memory_pgfault']>0)|
                                    (df_node['node_memory_pgmajfault']>0)|
                                    (df_node['node_network_rx_dropped']>0)|
                                    (df_node['node_network_rx_errors']>0)|
                                    (df_node['node_network_tx_dropped']>0)|
                                    (df_node['node_network_tx_errors']>0)
                                   
                                    
                                )









filter_Condition_Pod = np.where( 
                                    (df_pod['pod_memory_failcnt']>0) |
                                    (df_pod['pod_memory_hierarchical_pgfault']>0) |
                                    (df_pod['pod_memory_hierarchical_pgmajfault']>0)|
                                    (df_pod['pod_memory_pgfault']>0)|
                                    (df_pod['pod_memory_pgmajfault']>0)|
                                    (df_pod['pod_network_rx_dropped']>0)|
                                    (df_pod['pod_network_rx_errors']>0)|
                                    (df_pod['pod_network_tx_dropped']>0)|
                                    (df_pod['pod_network_tx_errors']>0) |
                                    (df_pod['pod_status'] == 'Failed')
                                )




filter_Condition_Container = np.where( 
                                    (df_container['container_memory_failcnt']>0) |
                                    (df_container['container_memory_hierarchical_pgfault']>0) |
                                    (df_container['container_memory_hierarchical_pgmajfault']>0)|
                                    (df_container['container_memory_pgmajfault']>0)
                                    
                                )


In [ ]:
unhealthy_clusternames_cluster = df_cluster.loc[filter_Condition_Cluster]['ClusterName'].unique().tolist()
unhealthy_clusternames_node = df_node.loc[filter_Condition_Node]['ClusterName'].unique().tolist()
unhealthy_clusternames_pod = df_pod.loc[filter_Condition_Pod]['ClusterName'].unique().tolist()
unhealthy_clusternames_container = df_container.loc[filter_Condition_Container]['ClusterName'].unique().tolist()

In [ ]:
all_unhealthy_cluster_names = list(set(unhealthy_clusternames_cluster+unhealthy_clusternames_node+unhealthy_clusternames_pod+unhealthy_clusternames_container))

Now lets drop all of the clusters that we have doubts about the anomolous status of and start creating our training feature set

In [ ]:
df_node_non_anomalous = df_node[~df_node['ClusterName'].isin(all_unhealthy_cluster_names)]

In [ ]:
df_node_non_anomalous.to_parquet('non_anomalous_train.parquet')

Quick Sanity Checks

In [ ]:
df_node_non_anomalous.columns

In [ ]:
df_node_non_anomalous.ClusterName.nunique()

In [ ]:
df_node_non_anomalous.InstanceId.nunique()

In [ ]:
df_node_non_anomalous.shape